# Importing libraries and modules


In [15]:
import os # For work with directories
import cv2 # For reading images and resize them
import numpy as np
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

# Preparing data

In [24]:
listFolders = os.listdir('./data')
data = []
labels = []

for i, folder in enumerate(listFolders):
    imgs = os.listdir('./data/' + folder)
    for img in imgs:
        imgReaded = cv2.imread('./data/' + folder + '/' + img)
        image_resized = cv2.resize(imgReaded, (48, 48))
        image_resized = np.array(image_resized)
        image_resized = image_resized / 255.0  # Normalize pixel values to [0, 1]
        data.append(image_resized)

        # label
        label = np.zeros(len(listFolders))
        label[i] = 1
        labels.append(label)

data = np.array(data)
labels = np.array(labels)

(48, 48, 3)


# Split data and  Flatten the image arrays

In [28]:
data, labels = shuffle(data, labels, random_state=2)
x_train, x_test, y_train, y_test = train_test_split(data, labels, test_size=0.2)

x_train_flatten = x_train.reshape(x_train.shape[0], -1)
x_test_flatten = x_test.reshape(x_test.shape[0], -1)

# Creating Model

In [30]:
model = MLPClassifier(hidden_layer_sizes=(128, 64), activation='relu')
model.fit(x_train_flatten, y_train)

C:\Users\hp\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPClassifier(hidden_layer_sizes=(128, 64))

# Accuracy

In [19]:
y_pred = model.predict(x_test_flatten)
accuracy = accuracy_score(y_test.argmax(axis=1), y_pred.argmax(axis=1))
print("Accuracy:", accuracy)


Accuracy: 0.9847715736040609


# Predict

In [35]:
def load_and_preprocess_image(image_path):
    imgReaded = cv2.imread(image_path)
    image_resized = cv2.resize(imgReaded, (48, 48))
    image_resized = np.array(image_resized)
    image_resized = image_resized / 255.0
    return image_resized
def predict_emotion(image_path, model):
    image = load_and_preprocess_image(image_path)
    flattened_image = image.reshape(1, -1)  # Reshape to a 2D array
    emotion_probs = model.predict_proba(flattened_image)
    predicted_emotion_index = np.argmax(emotion_probs)
    
    # Define a list of emotion labels
    emotion_labels = ["Angry", "Contempt", "Disgust", "Fear", "Happy", "Sadness", "Surprise"]
    
    predicted_emotion = emotion_labels[predicted_emotion_index]
    return predicted_emotion

In [53]:
def load_and_preprocess_image(image_path):
    imgReaded = cv2.imread(image_path)
    image_resized = cv2.resize(imgReaded, (48, 48))
    image_resized = np.array(image_resized)
    image_resized = image_resized / 255.0
    return image_resized

def face_detection(image_path):
    image = cv2.imread(image_path)
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    faces = face_cascade.detectMultiScale(image, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    return faces

def predict_emotion(image_path, model):
    faces = face_detection(image_path)
    
    if len(faces) == 0:
        image = load_and_preprocess_image(image_path)
        flattened_image = image.reshape(1, -1)  # Reshape to a 2D array
        emotion_probs = model.predict_proba(flattened_image)
        predicted_emotion_index = np.argmax(emotion_probs)
    
        # Define a list of emotion labels
        emotion_labels = ["Angry", "Contempt", "Disgust", "Fear", "Happy", "Sadness", "Surprise"]
    
        predicted_emotion = emotion_labels[predicted_emotion_index]
        return predicted_emotion
    
    # Consider the first detected face
    (x, y, w, h) = faces[0]
    face_image = cv2.imread(image_path)[y:y+h, x:x+w]
    face_image = cv2.resize(face_image, (48, 48))
    face_image = face_image / 255.0
    
    flattened_face = face_image.reshape(1, -1)  # Reshape to a 2D array
    emotion_probs = model.predict_proba(flattened_face)
    predicted_emotion_index = np.argmax(emotion_probs)
    
    # Define a list of emotion labels
    emotion_labels = ["Angry", "Contempt", "Disgust", "Fear", "Happy", "Sadness", "Surprise"]
    
    predicted_emotion = emotion_labels[predicted_emotion_index]
    return predicted_emotion

In [64]:
predict_emotion('surprise.jpg',model)

'Surprise'